In [1]:
pip install transformers datasets accelerate torch sentencepiece


In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import torch


##### Error : Runtime : Dataset scripts are no longer supported.
                dataset = load_dataset("multi_news", split="test")
                print(dataset)

In [3]:
dataset = load_dataset(
    "cnn_dailymail",
    "3.0.0",
    split="test"
)

print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})


##### Error: AttributeError: 'DatasetDict' object has no attribute   'to_pandas'
              dataset.to_pandas()

In [4]:
dataset = dataset.train_test_split(test_size=0.2)

train_data = dataset["train"]
test_data = dataset["test"]


In [5]:
df = dataset["train"].select(range(1000)).to_pandas()
df

,article,highlights,id
0,Victoria Beckham did it. So did Kim Kardashian...,Harper Beckham and North West have both attend...,9c531fb73eb3130322cd80b669c8bb5baa886e7e
1,Tony Blair has said he does not want to retire...,The former Prime Minister claimed he has 'deca...,207923621d28e67c9c5eb210a40c5d3848863ced
2,An aspiring 21-year-old rapper was shot dead w...,"Keaway Lafonz Ivy, known as Kealo, was shot an...",622c9c05a1b5cd5d0016826be24b9efad17ce009
3,Angelina Jolie may have made the character wor...,Bonnie's skill at playing Call Of Duty has gai...,0a9b7a7f8ba366adcec4a87971f4f258780008c2
4,Malcolm Brabant spent two years in and out of ...,BBC foreign correspondent Malcolm Brabant had ...,f102adff48f11eb326dd9c63268af64800d6fd07
...,...,...,...
995,Many of us enjoy a chilled glass of wine or a ...,Fifth of young people don't remember how they ...,59f86a0404e249e9f52ae260d011486215eea8d4
996,An eight-grade girl accused of conspiring with...,The defense petition to return Jamie Silvonek ...,01ec3bd760e4062d4e9e18a99238854bcd008ee9
997,(CNN)The head of the Libyan Army has rejected ...,Head of Libyan army tells CNN Libyan authoriti...,fd54c50ca32ef5b7cc8bb51ec9985eb1acbf36b9
998,The Brazilian man shot dead alongside Australi...,"Brazilian national Rodrigo Gularte, 42, was a ...",3aabf9cb0649b3c6dbd6cb03a86a00783217b427


In [6]:
from transformers import AutoTokenizer

In [7]:
model_checkpoint = "t5-small" # from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
prefix = "summarize"
def preprocess_function(examples):
    inputs = [prefix + article for article in examples["article"]]

    model_inputs = tokenizer(
        inputs,
        max_length=512,#1024
        truncation=True,
        #padding="max_length"
        padding=True
    )

    labels = tokenizer(
        text = examples["highlights"],
        max_length=128,
        truncation=True,
        #padding="max_length"
        padding=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [9]:
# Mapping
tokenized_train = train_data.map(
    preprocess_function,
    batched=True,
    remove_columns=train_data.column_names
)

tokenized_test = test_data.map(
    preprocess_function,
    batched=True,
    remove_columns=test_data.column_names
)


Map:   0%|          | 0/9192 [00:00<?, ? examples/s]

Map:   0%|          | 0/2298 [00:00<?, ? examples/s]

In [10]:
#Modelling : M1.M2,M3,M4
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# M1 :
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model_checkpoint
)


In [11]:
# M2 :
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
# M3 :
training_args = Seq2SeqTrainingArguments(
    # output_dir="./t5_text_summarizer",
    output_dir="./results",
    # eval_strategy="steps",
    eval_strategy="epoch",
    per_device_train_batch_size=1,#10
    per_device_eval_batch_size=1,#10
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    save_total_limit=3,
    dataloader_pin_memory=False,
    fp16=torch.cuda.is_available(),
    report_to="none"
)


In [13]:
# M4 :
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)


/tmp/ipython-input-4017322724.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
# Model Training
trainer.train()


Epoch,Training Loss,Validation Loss
1,1.231400,1.115127


TrainOutput(global_step=9192, training_loss=1.3221641993709394, metrics={'train_runtime': 1121.5302, 'train_samples_per_second': 8.196, 'train_steps_per_second': 8.196, 'total_flos': 1244061839130624.0, 'train_loss': 1.3221641993709394, 'epoch': 1.0})

In [15]:
trainer.save_model("t5-text-summarizer")
tokenizer.save_pretrained("t5-text-summarizer")


('t5-text-summarizer/tokenizer_config.json',
 't5-text-summarizer/special_tokens_map.json',
 't5-text-summarizer/spiece.model',
 't5-text-summarizer/added_tokens.json',
 't5-text-summarizer/tokenizer.json')

In [16]:
def summarize_text(text, max_length=120):
    inputs = tokenizer(
        "summarize: " + text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sample_text = """
Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, reshaping industries, economies, and everyday life. At its core, AI refers to the ability of machines to simulate human intelligence through learning, reasoning, problem-solving, and decision-making. From virtual assistants and recommendation systems to autonomous vehicles and medical diagnostics, AI applications are becoming increasingly integrated into modern society. Organizations across sectors are leveraging AI to automate repetitive tasks, analyze massive volumes of data, and uncover insights that were previously difficult or impossible to obtain through traditional methods.
In the business world, AI is driving efficiency and innovation by enabling predictive analytics, customer behavior modeling, and intelligent automation. Companies use AI-powered chatbots to enhance customer support, machine learning algorithms to optimize supply chains, and data-driven models to improve strategic planning. In healthcare, AI systems assist doctors by analyzing medical images, predicting disease risks, and supporting personalized treatment plans. These advancements have the potential to improve patient outcomes, reduce costs, and increase accessibility to quality healthcare services.
Despite its many benefits, the rapid growth of AI also raises significant ethical, social, and economic concerns. Issues such as data privacy, algorithmic bias, and transparency in decision-making have become major topics of discussion among policymakers and researchers. There is growing concern about job displacement as automation replaces certain roles, highlighting the need for reskilling and upskilling the workforce. Governments and regulatory bodies are increasingly working to establish guidelines that ensure responsible and fair use of AI technologies.
Looking ahead, the future of AI depends on balancing innovation with ethical responsibility. Continued research, collaboration between industry and academia, and strong governance frameworks will be essential to ensure that AI benefits society as a whole. When developed and deployed thoughtfully, artificial intelligence has the potential to address some of the world’s most complex challenges, from climate change and healthcare to education and sustainable development.
"""

print("Generated Summary:")
print(summarize_text(sample_text))


Generated Summary:
AI is driving efficiency and innovation by enabling predictive analytics, customer behavior modeling, and intelligent automation. Organizations across sectors are leveraging AI to automate repetitive tasks, analyze massive volumes of data, and uncover insights that were previously difficult or impossible to obtain.


##### Generated Summary:
AI is driving efficiency and innovation by enabling predictive analytics, customer behavior modeling, and intelligent automation. Organizations across sectors are leveraging AI to automate repetitive tasks, analyze massive volumes of data, and uncover insights that were previously difficult or impossible to obtain.